A World folder maintains the following files and subfolders:
* game_settings.json
* zero or more agent folders with name \<agent_name\> of your choice:
    1. \<agent_name\>.pth
    1. fig.png
    1. log.txt
    1. performance.csv
    1. all_rows.json
    1. old_\<agent_name\>.pth [HOLD]

The game_settings defines the game that all the agents will be playing.
It should not be modified once any agent is created.

In [ ]:
%pwd

In [ ]:
import os

import sys
import subprocess
import pkg_resources

import pathlib
import glob

import argparse
import numpy as np
import torch
import torch.nn as nn
from copy import deepcopy
from datetime import datetime
from pytz import timezone
from collections import OrderedDict
import more_itertools as mit
import pandas as pd
import json
import random
import re # regular expression

from enum import Enum 

from traitlets.traitlets import HasTraits, Int, Unicode, default

from typing import List, Set, Dict, Tuple, Any

import importlib

# Some folder functions

In [ ]:
def get_absolute_path_for_folder(folder): # for google colab
    result = None
    search_root = '/content' # for google colab
    possible_local_path = glob.glob(f'{search_root}/**/{folder}', recursive=True)
    if len(possible_local_path) == 1:
        result = pathlib.Path(possible_local_path[0]).resolve()
    return result

def enter_folder(folder:str, can_mkdir = True):
    absolute_path = get_absolute_path_for_folder(folder=folder)
    if absolute_path:
        %cd {absolute_path}
    elif can_mkdir:
        os.mkdir(folder)
        %cd {folder}
    else:
        print(f'No such folder: {folder}')

# Enter Gin_Rummy_Universe

In [ ]:
enter_folder('Gin_Rummy_Universe')

# Import Holoviz panel

In [ ]:
if 'panel' not in [pkg.key for pkg in pkg_resources.working_set]:
    !pip install panel
else:
    print('panel already installed')

from bokeh.core.validation import check

import panel as pn
from panel.interact import interact, fixed
from panel import widgets
import param

from panel.layout.gridstack import GridStack

# Install pylibcheck

In [ ]:
!pip install pylibcheck
import pylibcheck

# Import rlcard

In [ ]:
# if 'rlcard' in [pkg.key for pkg in pkg_resources.working_set]:
#     print('rlcard is already installed')
# else:
#     !pip install rlcard
#     print('rlcard has been installed')

if pylibcheck.checkPackage("rlcard"):
     print('rlcard is already installed')
else:
    !pip install rlcard
    print('rlcard has been installed')

In [ ]:
import rlcard
from rlcard.agents import RandomAgent
from rlcard.utils import get_device, set_seed, tournament, reorganize, Logger, plot_curve
from rlcard.agents import DQNAgent
from rlcard.models.gin_rummy_rule_models import GinRummyNoviceRuleAgent

from rlcard.agents.dqn_agent import Memory
from rlcard.agents.dqn_agent import Estimator

from rlcard.games.gin_rummy.player import GinRummyPlayer
from rlcard.games.gin_rummy.game import GinRummyGame
from rlcard.games.gin_rummy.utils.action_event import ActionEvent, DiscardAction
from rlcard.games.gin_rummy.utils.action_event import KnockAction, GinAction, DeclareDeadHandAction

from rlcard.games.gin_rummy.utils.scorers import GinRummyScorer
from rlcard.games.gin_rummy.utils.settings import Setting, Settings, DealerForRound

from rlcard.games.gin_rummy.utils.action_event import draw_card_action_id, pick_up_discard_action_id

import rlcard.games.gin_rummy.utils.utils as utils
import rlcard.games.gin_rummy.utils.melding as melding

from rlcard.games.gin_rummy.utils.thinker import Thinker

from rlcard.games.base import Card

from rlcard.envs.gin_rummy import GinRummyEnv

from rlcard.agents import NFSPAgent

import rlcard as rlcard # NOTE: don't know why this is needed; don't know why it has to be placed last

# Import from gin_rummy_lib

In [ ]:
def append_gin_rummy_lib_to_sys_path():
    absolute_path = get_absolute_path_for_folder('gin_rummy_lib')
    if absolute_path:
        absolute_path_str = str(absolute_path)
        if not absolute_path_str in sys.path:
            sys.path.append(absolute_path_str)
    else:
        abs_folder = pathlib.Path('gin-rummy-lib/').resolve()
        abs_path = abs_folder / 'gin_rummy_lib'
        if not os.path.exists(abs_folder):
            print(f'Cloning gin-rummy-lib to {abs_folder}')
            !git clone https://github.com/billh0420/gin-rummy-lib.git '{abs_folder}'
        abs_path_str = str(abs_path)
        if not abs_path_str in sys.path:
            sys.path.append(abs_path_str)

append_gin_rummy_lib_to_sys_path()
sys.path

In [ ]:
from util import to_int_list
from util import sortByRankBySuit
from pane.GameSettingsPane import GameSettingsPane
from DQNAgentConfig import DQNAgentConfig
from RLTrainerConfig import RLTrainerConfig
from RLTrainer import RLTrainer
from GinRummyScorer230402 import GinRummyScorer230402
from GinRummyRookie01RuleAgent import GinRummyRookie01RuleAgent
from GameMaker import GameMaker
from GinRummyGameMaker import GinRummyGameMaker
from GameObserver import GameObserver
from GameReviewer import GameReviewer
from World import World
from pane.DQNAgentPane import DQNAgentPane
from pane.RLTrainerPane import RLTrainerPane
from pane.ReviewPlayWindow import ReviewPlayWindow
from pane.WorldRLTrainerSettingsView import WorldRLTrainerSettingsView
from pane.TrainingResultsWindow import TrainingResultsWindow
from pane.ReviewMatchGamesWindow import ReviewMatchGamesWindow

from pane.WorldDQNAgentConfigWindow import WorldDQNAgentConfigWindow

from util import get_current_time

In [ ]:
pn.extension(raw_css=["""
    div.orange_border_table + table * {
        border: 1px solid orange;
    }
"""])

pn.extension(raw_css=["""
    div.special_table + table * {
        border: 1px solid orange;
        padding-left: 8px;
        padding-right: 8px;
    }
"""])

css_log_widget_box = '''
    .bk.log-widget-box {
        color: #ffffff;
        background: #000000;
        border: 1px black solid;
    }
'''
pn.extension(raw_css=[css_log_widget_box])

pn.extension('tabulator')

In [ ]:
# Need to do the following, but I don't know where to place it. Putting it earlier doesn't seem to always work.
pn.extension() # required for panels to be displayed in Jupyter notebook

# Final Version (Keep) 230407

In [ ]:
# Step 1 (one time only)
# Create an empty directory for your world.
# For example, create a directory with the name "Gin_Rummy_World"
# This directory will hold all the files that are used in your training sessions.
# This is done one time only.
# Make this your working directory.

enter_folder('Gin_Rummy_World')

In [ ]:
# Step 1.1
# Show game_maker_config for this world.
# The file game_maker_config.json will be created if it does not exists.
# Honor system: you can change the contents of this file as long as the first training agent is not created.

def show_game_maker_config(): # hard-coded in notebook
    game_maker_config = {'selection': 'win_or_lose', 'max_move_count': 50} # hard_coded
    print(f'--- Gin Rummy Game Maker Config ---')
    file_path = 'game_maker_config.json' # cannot change this
    if not os.path.exists(file_path):
        pd.Series(game_maker_config).to_json(path_or_buf=file_path, orient='index')
    for key, value in pd.read_json(path_or_buf=file_path, typ='series', orient='index').items():
        print(f'{key}: {value}')

show_game_maker_config()

In [ ]:
# Step 2 (always done to get handle on the world with your choice of name.)

# Open world.
#
# This is done each time you want to do something with your agents.
# Here the world is called 'gin_rummy_world'. You can call it simply 'world' if you wish.
#
# If you do not specify the world_dir, then the current working directory is used.
# If you specify its directory, then it must exists else a crash will result.
# Example: gin_rummy_world = World(world_dir='../results_gin_rummy_dqn')
#
# Note: The world variable is the only variable that is allocated in these steps.
#       I am trying to minimize the creation of other named variables (e.g. views and windows).
#
# The world_dir will hold the following:
#   1) The file game_maker_config.json (Created in step 1.1).
#   2) The directory whose name is the agent name (It will be automatically created when the agent is created).
#       This directory holds the following files:
#       - the agent pth file (modified by training it in step 6);
#       - the file log.txt created by play_train_match in step 6;
#       - the file fig.png created by play_train_match in step 6;
#       - the file performance.csv created by play_train_match in step 6.

def get_gin_rummy_world():
    game_maker = GinRummyGameMaker()
    gin_rummy_world = World(game_maker=game_maker) # Using current directory as world_dir
    return gin_rummy_world

gin_rummy_world = get_gin_rummy_world()

In [ ]:
# Step 3 (optional)
# Use GameSettingsPane to view game settings (optional).
# After you create your first training agent in the next step, you should not change the game settings.
# The agents assume that they are playing with the same game settings that existed when the first agent was created.
# That is, all agents are playing with the same rules for the game.

GameSettingsPane(world=gin_rummy_world)

In [ ]:
# Step 4 (optional)
# Use WorldDQNAgentConfigWindow to create your training agent (one time per agent).
# You adjust the settings and hit the "Create DQN Agent" button.
# If an agent with the chosen name already exists, nothing happens.
# You will get a message on whether you were successful or not (not implemented).

WorldDQNAgentConfigWindow(world=gin_rummy_world)

In [ ]:
# Step 5 (optional)
# Use WorldRLTrainerSettingsView to set trainer settings (multiple times except for changing algorithm)

WorldRLTrainerSettingsView(world=gin_rummy_world).view

In [ ]:
# Step 6 (optional)
# Run training session (long)

#%%time
gin_rummy_world.play_train_match(num_episodes=20)

In [ ]:
# Step 7 (optional)
# Use TrainingResultsWindow to see results of training (optional)
# Repeat steps 5, 6, 7 as often as you want. Of course, step 5 and 7 will be optional.

TrainingResultsWindow(world=gin_rummy_world)

In [ ]:
# Step 8 (optional)
# Use ReviewMatchGamesWindow to review how well the dqn_agent is doing (optional)
# Note that 0 <= max_review_episodes <= 1000.
# Repeat any or several of these steps.

ReviewMatchGamesWindow(world=gin_rummy_world)